In [5]:
from transformers import BertTokenizer,BertForSequenceClassification,Trainer,TrainingArguments
from datasets import load_dataset
import pandas as pd

In [6]:
dataset= load_dataset('imdb')

In [7]:
local_model_path=r"C:\Users\balap\for jupyter note book\BERT\bert_local"

In [9]:
tokenzier=BertTokenizer.from_pretrained(local_model_path)
def tokenize_function(examples):
    return tokenizer(examples['text'],padding='max_length',truncation=True)
tokenize_dataset= dataset.map(tokenize_function, batched=True)

In [9]:
model= BertForSequenceClassification.from_pretrained(local_model_path,num_labels=2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at C:\Users\balap\for jupyter note book\BERT\bert_local and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
trainig_arg=TrainingArguments(
    output_dir='./results',
    eval_strategy ='epoch',
    learning_rate=0.00002,
    per_device_train_batch_size= 8,
    num_train_epochs=1,
    weight_decay= 0.01,
)

    


In [20]:
trainer= Trainer(
    model=model,
    args= trainig_arg,
    train_dataset= tokenize_dataset['train'].shuffle(seed=42).select(range(500)),
    eval_dataset= tokenize_dataset['test'].shuffle(seed=42).select(range(100)),
)
                                                    
                                            


In [22]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.469704


C:\Users\balap\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\balap\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


TrainOutput(global_step=63, training_loss=0.5704627264113653, metrics={'train_runtime': 1081.0311, 'train_samples_per_second': 0.463, 'train_steps_per_second': 0.058, 'total_flos': 131555527680000.0, 'train_loss': 0.5704627264113653, 'epoch': 1.0})

In [39]:
inputs= tokenizer("This movie is wwaste ",return_tensors='pt')
outputs=model(**inputs)
logits=outputs.logits
predicted_class=logits.argmax().item()
print("predicted class:","positive" if predicted_class==1 else "negative")

predicted class: positive


In [59]:
# Save model
model.save_pretrained("./bert_sentiment_model")
# Save tokenizer
tokenizer.save_pretrained("./bert_sentiment_model")


('./bert_sentiment_model\\tokenizer_config.json',
 './bert_sentiment_model\\special_tokens_map.json',
 './bert_sentiment_model\\vocab.txt',
 './bert_sentiment_model\\added_tokens.json')